In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import gc
import xgboost as xgb
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression
import pickle
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor

In [9]:
transactions    = pd.read_csv('sales_train.csv.gz')
items           = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
test            = pd.read_csv('test.csv.gz')

In [10]:
INDEX_COLS = ['item_id', 'shop_id', 'date_block_num']

In [11]:
transactions = pd.merge(transactions, items, on='item_id', how='left')
transactions = transactions.drop('item_name', axis=1)
transactions = transactions[transactions['date_block_num'] > 12]
transactions.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
1366911,01.02.2014,13,27,15242,699.0,1.0,63
1366912,25.02.2014,13,27,15200,299.0,1.0,69
1366913,19.02.2014,13,27,15279,799.0,1.0,63
1366914,26.02.2014,13,27,15202,299.0,1.0,69
1366915,01.02.2014,13,27,14888,549.0,1.0,55


In [ ]:
len(transactions)

In [ ]:
np.append(test_item_ids,train_item_ids)

In [34]:
train_item_ids = transactions['item_id'].unique()
train_shop_ids = transactions['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = transactions['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))

In [108]:
items[['item_id', 'item_category_id']].to_dict('dict')

{'item_id': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49,
  50: 50,
  51: 51,
  52: 52,
  53: 53,
  54: 54,
  55: 55,
  56: 56,
  57: 57,
  58: 58,
  59: 59,
  60: 60,
  61: 61,
  62: 62,
  63: 63,
  64: 64,
  65: 65,
  66: 66,
  67: 67,
  68: 68,
  69: 69,
  70: 70,
  71: 71,
  72: 72,
  73: 73,
  74: 74,
  75: 75,
  76: 76,
  77: 77,
  78: 78,
  79: 79,
  80: 80,
  81: 81,
  82: 82,
  83: 83,
  84: 84,
  85: 85,
  86: 86,
  87: 87,
  88: 88,
  89: 89,
  90: 90,
  91: 91,
  92: 92,
  93: 93,
  94: 94,
  95: 95,
  96: 96,
  97: 97,
  98: 98,
  99: 99,
  100: 10

In [46]:
combinations = []
for shop in all_shop_ids:
    #get all article ids ever associated to this shop
    train_ids = transactions[transactions['shop_id'] == shop]['item_id'].unique()
    test_ids = test[test['shop_id'] == shop]['item_id'].unique()
    all_shop = np.unique(np.append(train_ids, test_ids))
    all_shop_combo = [[item, shop, block] for item in all_shop for block in train_blocks]
    for combo in all_shop_combo:
        combinations.append(combo)

In [45]:
len(combinations)

7722666

In [47]:
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['item_id', 'shop_id', 'date_block_num'])

In [48]:
len(all_combos)

7722666

In [110]:
all_combos.head()

,item_id,shop_id,date_block_num,item_category_id
0,0,54,13,40
1,0,54,14,40
2,0,54,15,40
3,0,54,16,40
4,0,54,17,40


In [109]:
all_combos = pd.merge(all_combos, items[['item_id', 'item_category_id']], on='item_id', how='left')

In [50]:
transactions['y'] = transactions.groupby(['date_block_num','shop_id','item_id'])['item_cnt_day'].transform(np.sum).clip(0,40)

In [51]:
gc.collect()

transactions['item_cnt_sum'] = transactions.groupby(['item_id', 'date_block_num'])['item_cnt_day'].transform(np.sum)
transactions['item_cnt_mean'] = transactions.groupby(['item_id', 'date_block_num'])['item_cnt_day'].transform(np.mean)
transactions['item_price_mean'] = transactions.groupby(['item_id', 'date_block_num'])['item_price'].transform(np.mean)


transactions['shop_cnt_sum'] = transactions.groupby(['shop_id', 'date_block_num'])['item_cnt_day'].transform(np.sum)
transactions['shop_cnt_mean'] = transactions.groupby(['shop_id', 'date_block_num'])['item_cnt_day'].transform(np.mean)
transactions['shop_price_mean'] = transactions.groupby(['shop_id', 'date_block_num'])['item_price'].transform(np.mean)

transactions['category_cnt_sum'] = transactions.groupby(['item_category_id', 'date_block_num'])['item_cnt_day'].transform(np.sum)
transactions['category_cnt_mean'] = transactions.groupby(['item_category_id', 'date_block_num'])['item_cnt_day'].transform(np.mean)
transactions['category_price_mean'] = transactions.groupby(['item_category_id', 'date_block_num'])['item_price'].transform(np.mean)

In [52]:
len(transactions)

868080

In [53]:
transactions = transactions.drop_duplicates(INDEX_COLS)

In [54]:
len(transactions)

868080

In [101]:
transactions['item_category_id'].nunique()

78

In [111]:
training = pd.merge(all_combos,transactions,on=['item_id', 'shop_id', 'date_block_num'],how='left')

In [121]:
training.drop('item_category_id_y', inplace=True, axis=1)

KeyError: "labels ['item_category_id_y'] not contained in axis"

In [120]:
training.columns

Index(['item_id', 'shop_id', 'date_block_num', 'item_category_id_x', 'date',
       'item_price', 'item_cnt_day', 'y', 'item_cnt_sum', 'item_cnt_mean',
       'item_price_mean', 'shop_cnt_sum', 'shop_cnt_mean', 'shop_price_mean',
       'category_cnt_sum', 'category_cnt_mean', 'category_price_mean'],
      dtype='object')

In [126]:
training.rename(columns={'item_category_id_x': 'item_category_id'}, inplace=True)

In [127]:
training.head()

,item_id,shop_id,date_block_num,item_category_id,date,item_price,item_cnt_day,y,item_cnt_sum,item_cnt_mean,item_price_mean,shop_cnt_sum,shop_cnt_mean,shop_price_mean,category_cnt_sum,category_cnt_mean,category_price_mean
0,0,54,13,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,54,14,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,54,15,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,54,16,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,54,17,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
training[['item_id', 'shop_id', 'item_category_id', 'date_block_num']].sample(100)

,item_id,shop_id,item_category_id,date_block_num
6785834,19341,15,49,33
1345246,4117,39,58,20
5993531,17027,7,55,18
5491649,15770,57,38,15
2853537,8625,59,40,28
7074201,20252,31,40,28
3040754,9203,4,61,30
7402553,21265,28,61,24
5150924,15031,58,49,15
3217396,9870,5,40,20


In [129]:
training['y'] = training['y'].fillna(0)

In [130]:
transactions[(transactions['item_id'] == 15266) & (transactions['shop_id'] == 5)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,y,item_cnt_sum,item_cnt_mean,item_price_mean,shop_cnt_sum,shop_cnt_mean,shop_price_mean,category_cnt_sum,category_cnt_mean,category_price_mean
2332910,24.01.2015,24,5,15266,699.0,1.0,63,1.0,23.0,1.0,692.956522,810.0,1.033163,1120.071696,1476.0,1.035789,890.800954
2527492,21.03.2015,26,5,15266,699.0,1.0,63,1.0,21.0,1.0,682.857143,702.0,1.035398,833.320575,1359.0,1.026435,930.227372


In [131]:
training[(training['item_id'] == 15266) & (training['shop_id'] == 5)]

,item_id,shop_id,date_block_num,item_category_id,date,item_price,item_cnt_day,y,item_cnt_sum,item_cnt_mean,item_price_mean,shop_cnt_sum,shop_cnt_mean,shop_price_mean,category_cnt_sum,category_cnt_mean,category_price_mean
5266191,15266,5,13,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266192,15266,5,14,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266193,15266,5,15,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266194,15266,5,16,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266195,15266,5,17,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266196,15266,5,18,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266197,15266,5,19,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266198,15266,5,20,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266199,15266,5,21,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266200,15266,5,22,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
len(training)

7722666

In [133]:
training = training.sample(frac=1)

In [134]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

training_int = training.select_dtypes(include=['int'])
converted_int = training_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(training_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([training_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

58.92 MB
58.92 MB


,before,after


In [135]:
training_float = training.select_dtypes(include=['float'])
converted_float = training_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(training_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([training_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

765.95 MB
412.43 MB


,before,after
float32,NaN,12.0
float64,12.0,NaN


In [136]:
optimized_training = training.copy()

optimized_training[converted_int.columns] = converted_int
optimized_training[converted_float.columns] = converted_float

print(mem_usage(training))
print(mem_usage(optimized_training))

1266.28 MB
912.76 MB


In [137]:
training = optimized_training
del optimized_training
gc.collect()

707

In [138]:
lag_columns = training.columns[7:]
lags = [1,2,3,6,12]

In [139]:
training.head()

,item_id,shop_id,date_block_num,item_category_id,date,item_price,item_cnt_day,y,item_cnt_sum,item_cnt_mean,item_price_mean,shop_cnt_sum,shop_cnt_mean,shop_price_mean,category_cnt_sum,category_cnt_mean,category_price_mean
5091495,14845,15,16,65,03.05.2014,229.0,2.0,2.0,48.0,1.92,227.167999,938.0,1.111374,702.385681,880.0,1.222222,322.151093
5099338,14866,25,26,37,11.03.2015,399.0,1.0,1.0,6.0,1.00,399.000000,2678.0,1.106154,840.005981,3439.0,1.040545,392.435455
1046784,3347,54,31,23,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1782576,5380,7,25,75,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4980844,14494,24,14,41,01.03.2014,148.0,1.0,1.0,4.0,1.00,135.500000,1005.0,1.062368,871.382324,616.0,1.011494,1080.159424


In [140]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

merge_columns = ['lagged_block','item_id','shop_id']

for lag in lags:
    print(lag)
    lagged = transactions.copy()
    lagged.rename(columns={'date_block_num':'lagged_block'},inplace=True)
    training['lagged_block'] = training['date_block_num'] - lag
    lagged_names = [lagged_name(c,lag) for c in lag_columns]
    lag_mapping = dict(zip(lag_columns, lagged_names))
    lagged.rename(columns=lag_mapping,inplace=True)
    training = pd.merge(training,lagged[lagged_names+merge_columns],on=merge_columns,how='left')
    del lagged
    gc.collect()

1
2
3
6
12


In [141]:
len(training)

7722666

In [142]:
training.sample(10)

,item_id,shop_id,date_block_num,item_category_id,date,item_price,item_cnt_day,y,item_cnt_sum,item_cnt_mean,...,y_lag_12,item_cnt_sum_lag_12,item_cnt_mean_lag_12,item_price_mean_lag_12,shop_cnt_sum_lag_12,shop_cnt_mean_lag_12,shop_price_mean_lag_12,category_cnt_sum_lag_12,category_cnt_mean_lag_12,category_price_mean_lag_12
2688817,1384,22,29,30,08.06.2015,248.0,1.0,1.0,27.0,1.173913,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1646559,20072,12,17,37,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7127129,11172,28,24,40,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6895533,3787,50,14,55,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3024749,16533,42,30,37,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6464168,16174,34,24,65,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4346929,16560,44,14,40,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6199640,2642,53,24,55,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4815175,12705,12,18,40,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3463591,14112,26,15,37,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
training['date_block_num'].unique()

array([26, 16, 24, 15, 23, 33, 30, 21, 32, 25, 31, 20, 19, 18, 22, 17, 29,
       28, 27, 14, 13], dtype=int64)

In [ ]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training[(training['item_id'] == 30) & (training['shop_id'] == 30)].sort_values(by='date_block_num')

In [143]:
training.drop(columns=['y_lag_1', 'y_lag_2', 'y_lag_3', 'y_lag_6', 'y_lag_12', 'lagged_block'],inplace=True)

In [144]:
import pickle as pickle

pickle.dump(training, open( "training", "wb"), protocol=4)

#training = pickle.load( open( "training", "rb" ) )

In [145]:
val_indices = training[(training['date_block_num'] == 33)].index

In [146]:
lgbm_features = training.columns[17:]
lgbm_features = np.append( ['item_id', 'shop_id', 'item_category_id', 'date_block_num'], lgbm_features)
lgbm_features

array(['item_id', 'shop_id', 'item_category_id', 'date_block_num',
       'item_cnt_sum_lag_1', 'item_cnt_mean_lag_1',
       'item_price_mean_lag_1', 'shop_cnt_sum_lag_1',
       'shop_cnt_mean_lag_1', 'shop_price_mean_lag_1',
       'category_cnt_sum_lag_1', 'category_cnt_mean_lag_1',
       'category_price_mean_lag_1', 'item_cnt_sum_lag_2',
       'item_cnt_mean_lag_2', 'item_price_mean_lag_2',
       'shop_cnt_sum_lag_2', 'shop_cnt_mean_lag_2',
       'shop_price_mean_lag_2', 'category_cnt_sum_lag_2',
       'category_cnt_mean_lag_2', 'category_price_mean_lag_2',
       'item_cnt_sum_lag_3', 'item_cnt_mean_lag_3',
       'item_price_mean_lag_3', 'shop_cnt_sum_lag_3',
       'shop_cnt_mean_lag_3', 'shop_price_mean_lag_3',
       'category_cnt_sum_lag_3', 'category_cnt_mean_lag_3',
       'category_price_mean_lag_3', 'item_cnt_sum_lag_6',
       'item_cnt_mean_lag_6', 'item_price_mean_lag_6',
       'shop_cnt_sum_lag_6', 'shop_cnt_mean_lag_6',
       'shop_price_mean_lag_6', 'categor

In [147]:
categorical_features_indices = [0,1,2,3]

In [148]:
training[['item_id', 'shop_id', 'item_category_id', 'date_block_num']].sample(100)

,item_id,shop_id,item_category_id,date_block_num
1382684,18253,31,57,19
3307860,6286,38,55,28
371249,3558,21,24,27
7267720,12117,5,16,16
1087028,21046,42,61,32
2104900,1101,50,57,25
7115008,9335,14,70,24
478440,8653,33,40,30
951962,3598,42,58,29
2614389,14956,48,47,26


In [149]:
x = training
y = training['y']


x_train = x[~x.index.isin(val_indices)]
y_train = y[~y.index.isin(val_indices)]

x_val = x[x.index.isin(val_indices)]
y_val = y[y.index.isin(val_indices)]

In [ ]:
gc.collect()
lgtrain = lgbm.Dataset(x_train[lgbm_features[best_features_indices]], label=y_train)
lgval = lgbm.Dataset(x_val[lgbm_features[best_features_indices]], label=y_val)



#[0.00542047893814942, 29, 24, 0.39949465609514856, 1, 0.67943500, 10]
params = {
        "num_threads": 16,
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        #"max_bin": 10,#default 255
        #"num_leaves": 10, #default 31
        "bagging_fraction": 0.3,
        "bagging_freq": 1,
        "min_data_in_leaf": 50000,
        #"feature_fraction": 0.5,
        #"min_gain_to_split": 1,
       # "lambda_l1": 100,
        #"lambda_l2": 100,
        #"max_depth": 3, #default -1
        #"min_gain_to_split": 10,
        "learning_rate" : 0.02,
        #"histogram_pool_size": 1000,
        #"categorical_column": [0,1,2,3,4]
}


evals_result = {}
model_lgb = lgbm.train(params, lgtrain, 1000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=100, 
                      evals_result=evals_result)



In [150]:
cb_model = CatBoostRegressor(iterations=1000,
                             learning_rate=0.03,
                             eval_metric='RMSE',
                             #thread_count=16,
                             task_type = "GPU",
                             use_best_model=True,
                             #l2_leaf_reg = 1000,
                            od_type = "Iter",
                            od_wait = 100,
                             random_strength = 10,
                             bagging_temperature = 1,
                             random_seed = 23)


cb_model.fit(x_train[lgbm_features], y_train, cat_features=categorical_features_indices,
             eval_set=(x_val[lgbm_features],y_val),
             #cat_features=categorical_features_pos,         
             verbose=True)

0:	learn: 0.4573279	test: 0.4396969	best: 0.4396969 (0)	total: 943ms	remaining: 15m 42s
1:	learn: 0.4542370	test: 0.4381210	best: 0.4381210 (1)	total: 1.67s	remaining: 13m 52s
2:	learn: 0.4511619	test: 0.4368296	best: 0.4368296 (2)	total: 2.7s	remaining: 14m 58s
3:	learn: 0.4481708	test: 0.4358895	best: 0.4358895 (3)	total: 3.51s	remaining: 14m 34s
4:	learn: 0.4454407	test: 0.4347604	best: 0.4347604 (4)	total: 4.37s	remaining: 14m 29s
5:	learn: 0.4426103	test: 0.4340099	best: 0.4340099 (5)	total: 5.26s	remaining: 14m 31s
6:	learn: 0.4401171	test: 0.4329412	best: 0.4329412 (6)	total: 6.12s	remaining: 14m 28s
7:	learn: 0.4376865	test: 0.4318109	best: 0.4318109 (7)	total: 6.97s	remaining: 14m 23s
8:	learn: 0.4353109	test: 0.4309027	best: 0.4309027 (8)	total: 8.09s	remaining: 14m 51s
9:	learn: 0.4329145	test: 0.4301838	best: 0.4301838 (9)	total: 8.81s	remaining: 14m 32s
10:	learn: 0.4308460	test: 0.4293492	best: 0.4293492 (10)	total: 9.7s	remaining: 14m 32s
11:	learn: 0.4286747	test: 0.428

92:	learn: 0.3819859	test: 0.4105552	best: 0.4105552 (92)	total: 1m 23s	remaining: 13m 30s
93:	learn: 0.3818238	test: 0.4105400	best: 0.4105400 (93)	total: 1m 23s	remaining: 13m 29s
94:	learn: 0.3815909	test: 0.4105346	best: 0.4105346 (94)	total: 1m 25s	remaining: 13m 31s
95:	learn: 0.3813348	test: 0.4104727	best: 0.4104727 (95)	total: 1m 26s	remaining: 13m 30s
96:	learn: 0.3812680	test: 0.4104496	best: 0.4104496 (96)	total: 1m 26s	remaining: 13m 28s
97:	learn: 0.3810614	test: 0.4104609	best: 0.4104496 (96)	total: 1m 28s	remaining: 13m 30s
98:	learn: 0.3808781	test: 0.4104877	best: 0.4104496 (96)	total: 1m 28s	remaining: 13m 29s
99:	learn: 0.3807533	test: 0.4105211	best: 0.4104496 (96)	total: 1m 29s	remaining: 13m 29s
100:	learn: 0.3806234	test: 0.4105548	best: 0.4104496 (96)	total: 1m 30s	remaining: 13m 28s
101:	learn: 0.3805560	test: 0.4105389	best: 0.4104496 (96)	total: 1m 31s	remaining: 13m 27s
102:	learn: 0.3804439	test: 0.4105401	best: 0.4104496 (96)	total: 1m 32s	remaining: 13m 

181:	learn: 0.3746495	test: 0.4107177	best: 0.4102346 (127)	total: 2m 37s	remaining: 11m 48s
182:	learn: 0.3744598	test: 0.4107131	best: 0.4102346 (127)	total: 2m 38s	remaining: 11m 47s
183:	learn: 0.3743859	test: 0.4107299	best: 0.4102346 (127)	total: 2m 39s	remaining: 11m 48s
184:	learn: 0.3743476	test: 0.4105622	best: 0.4102346 (127)	total: 2m 40s	remaining: 11m 47s
185:	learn: 0.3742663	test: 0.4105690	best: 0.4102346 (127)	total: 2m 41s	remaining: 11m 46s
186:	learn: 0.3742459	test: 0.4105703	best: 0.4102346 (127)	total: 2m 42s	remaining: 11m 46s
187:	learn: 0.3742023	test: 0.4105682	best: 0.4102346 (127)	total: 2m 43s	remaining: 11m 44s
188:	learn: 0.3741180	test: 0.4105539	best: 0.4102346 (127)	total: 2m 44s	remaining: 11m 44s
189:	learn: 0.3740839	test: 0.4105365	best: 0.4102346 (127)	total: 2m 44s	remaining: 11m 42s
190:	learn: 0.3740550	test: 0.4105908	best: 0.4102346 (127)	total: 2m 45s	remaining: 11m 41s
191:	learn: 0.3740285	test: 0.4105925	best: 0.4102346 (127)	total: 2m 

In [ ]:
len(lgbm_features)

In [ ]:
print('Plotting feature importances...')
ax = lgbm.plot_importance(model_lgb, max_num_features=10)
plt.show()

In [ ]:
best_features_indices = np.argsort(model_lgb.feature_importance())[::-1][0:5]
lgbm_features[best_features_indices]

In [151]:
test_w_cat_ids = test.set_index('item_id').join(items.set_index('item_id'))
test_w_cat_ids['date_block_num'] = 34

In [152]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

merge_columns = ['lagged_block','item_id','shop_id']

for lag in lags:
    print(lag)
    lagged = transactions.copy()
    lagged.rename(columns={'date_block_num':'lagged_block'},inplace=True)
    test_w_cat_ids['lagged_block'] = test_w_cat_ids['date_block_num'] - lag
    lagged_names = [lagged_name(c,lag) for c in lag_columns]
    lag_mapping = dict(zip(lag_columns, lagged_names))
    lagged.rename(columns=lag_mapping,inplace=True)
    test_w_cat_ids = pd.merge(test_w_cat_ids,lagged[lagged_names+merge_columns],on=merge_columns,how='left')
    del lagged
    gc.collect()

1
2
3
6
12


In [28]:
test_w_cat_ids.fillna(0, inplace=True)

In [29]:
test_w_cat_ids.head()

,ID,item_id,shop_id,item_name,item_category_id,date_block_num,lagged_block,y_lag_1,item_cnt_sum_lag_1,item_cnt_mean_lag_1,...,y_lag_12,item_cnt_sum_lag_12,item_cnt_mean_lag_12,item_price_mean_lag_12,shop_cnt_sum_lag_12,shop_cnt_mean_lag_12,shop_price_mean_lag_12,category_cnt_sum_lag_12,category_cnt_mean_lag_12,category_price_mean_lag_12
0,2587,30,5,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,34,22,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0000,0.000000
1,7687,30,4,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,34,22,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0000,0.000000
2,12787,30,6,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,34,22,0.0,0.0,0.0,...,1.0,13.0,1.0,169.0,3150.0,1.379159,1262.87496,14815.0,1.0903,264.032958
3,17887,30,3,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,34,22,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0000,0.000000
4,22987,30,2,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,34,22,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0000,0.000000


In [ ]:
len(test_w_cat_ids)

In [ ]:
preds = model_lgb.predict(test_w_cat_ids[lgbm_features])
preds.clip(0,20,out=preds)

In [153]:
preds = cb_model.predict(test_w_cat_ids[lgbm_features])
preds.clip(0,20,out=preds)

array([0.12926339, 0.0639561 , 0.17324513, ..., 0.11329911, 0.14906362,
       0.07291897])

In [154]:
np.sum(preds)

22466.27820788094

In [155]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds.astype(int)
#submission['item_cnt_month'] = ensemble_preds.astype(int)


submission.to_csv('submission.csv', index=False)